In [1]:
from transformers import GemmaTokenizer, AutoModelForCausalLM
import torch

In [3]:
gpu = torch.device('cuda:0')

In [4]:
model_id = "google/codegemma-2b"
tokenizer = GemmaTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map=gpu, torch_dtype=torch.float16)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
prompt = "def hello_world():\n"

In [6]:
# pt stands for pytorch
inputs = tokenizer(prompt, return_tensors="pt").to(gpu)
inputs

{'input_ids': tensor([[     2,   1293,  25612, 235298,   9097,   4409,    108]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [7]:
outputs = model.generate(**inputs, max_new_tokens=100)
outputs

tensor([[     2,   1293,  25612, 235298,   9097,   4409,    108,    141,   1431,
            885,   4521, 235269,   3855,  47347,    108,     70,      1]],
       device='cuda:0')

In [8]:
print(tokenizer.decode(outputs[0]))

<bos>def hello_world():
    print("Hello, World!")
<|file_separator|><eos>


In [9]:
len(tokenizer.get_vocab())

256000

In [10]:
tokenizer.tokenize('sully')

['s', 'ully']

In [11]:
tokenizer.tokenize('cougar')

['cou', 'gar']

In [12]:
tokenizer.tokenize('höger')

['hö', 'ger']

In [13]:
tokenizer.tokenize('rechts')

['rechts']

In [14]:
tokenizer.tokenize('11')

['1', '1']

In [15]:
tokenizer.tokenize('mål')

['mål']

In [16]:
tokenizer.tokenize('🐝')

['🐝']

In [17]:
torch.argmax(torch.softmax(model(**inputs).logits[0][-1], 0))

tensor(141, device='cuda:0')

In [20]:
torch.softmax(model(**inputs).logits[0][-1], 0)

tensor([1.1449e-09, 7.6911e-06, 9.2773e-06,  ..., 7.0833e-08, 3.6330e-06,
        1.2187e-09], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [64]:
tokenizer.decode([141])

'    '

In [73]:
tokenizer.decode(model.generate(**inputs, num_beams=8, max_new_tokens=256)[0])

'<bos>def hello_world():\ndef hello_world():\n    print("Hello, World!")\n\nif __name__ == "__main__":\n    hello_world()\n<|file_separator|><eos>'

In [75]:
tokenizer.decode(model.generate(**inputs, do_sample=True, top_p=0.9)[0])

"<bos>def hello_world():\ndef hello_world():\n  print('Hello World!')\n"

In [77]:
tokenizer.decode(model.generate(**inputs, do_sample=True, top_k=3)[0])

'<bos>def hello_world():\ndef hello_world():\n    print("Hello, world!")'

In [1]:
tokenizer.decode(model.generate(**inputs, do_sample=True, top_p=0.9, temperature=0.1)[0])

NameError: name 'tokenizer' is not defined